# Automatic catchment delineation using python and postgis

In [1]:
#Loading required modules

import os
import shutil
import subprocess
import psycopg2 as db
import psycopg2.extras
from psycopg2 import sql
from encrypt import decryptCredentials,decryptString
from procedures import refreshProcedures
import yaml
import getpass

from pygments import highlight
from pygments.formatters import HtmlFormatter
from pygments.lexers import YamlLexer
from IPython.core.display import display,HTML

import json
import gmaps
import gmaps.geojson_geometries
import geojson

## Pre-requisites

The name of the geodatabase is *Basins_HWI*
A schema called _norway_ should be created beforehand in the geodatabase.

Both rivers and dems can be uploaded to the database to dynamically generated tables using code in this notebook.
This should only be done once since it is a time-consuming operation.

For future reference, these are some of the commands used to create the database:
```

cursor.execute('ALTER DATABASE \"Basins_HWI\" SET search_path=public, postgis, contrib,topology;')
cursor.execute('ALTER DATABASE \"Basins_HWI\" SET postgis.gdal_enabled_drivers = \'ENABLE_ALL\';')
cursor.execute('CREATE EXTENSION postgis;')
cursor.execute('CREATE EXTENSION postgis_topology;')
cursor.execute('ALTER DATABASE \"Basins_HWI\" SET search_path=public, postgis, contrib,topology;')
cursor.execute('ALTER DATABA \"Basins_HWI\" SET postgis.gdal_enabled_drivers = \'ENABLE_ALL\';')
cursor.execute('SELECT pg_reload_conf();')
cursor.execute('SET postgis.enable_outdb_rasters TO True;')
```

The operations required to organize the elevation and river data datasets necessary to perform the catchment delineation have been (partially) stored in the geodatabase as stored procedures. These stored procedures are defined in the _procedures.py_ file in this repository.


### Elevation data

Elevation data should be downloaded from Kartverket. This has not yet been done for all of Norway but for parts of it. It might be possible to use some automation but this has been done by pointing and clicking. 

10m resolution dem have been used.

Some remarks regarding raster upload:

* So far rasters have been uploaded for the following projections:
    * EPSG:32633
    * EPSG:32632

* Any additional projections are not automagically handled.

* When uploading rasters to a table, make sure the first uploaded raster has the largest extent. Othewise, strange things happen. I haven't figured out why. Also avoid underscores in the raster files that are uploaded. Tile the uploaded raster otherwise queries become slow.


The dems having a common projection were uploaded to separate tables in the _norway_ schema.
* _norway.dem32_ (for utm 32 data)
* _norway.dem33_ (for utm 33 data)

### River shapefiles

The river shapefiles was downloaded from NVE and uploaded geodatabase to _norway_ in a table called _norway.rivers _ .

### Postgis database

Postgresql 5.5 with Postgis 2.4 have been used.
Postgis might need to be explicitly enabled.

The name of the geodatabase is *Basins_HWI*


#### Connection credentials

The credentials necessary to connect to the database have been encrypted using the _cryptography_ package.
The encryption and decryption functions are defined in the _encrypt.py_ file of this repository.

### Taudem

The catchment delineation routines come from the [Taudem](http://hydrology.usu.edu/taudem/taudem5/index.html) package. The package's routines should be installed and available from the path.

## Catchment delineation

### Setting up credentials and connecting to database

In [2]:
#Setting up credentials for database access. These should have been previously encrypted
token = b'gAAAAABaVgNb96o6n1Kixc3fHKQWyEPN7jnJvXv_NJs65yjvJDqZZOH4w9aTyYJD28kx3iJr4EG0nsqTgxv_PRCOPKjkGPlQHycz8BuRTr25vETKiPAbLT28CJWLYLnWMllF_M1sGj_GErPOciHOQiraNUuo6IJMlVnUVMR5FvhP7YtqCKwtLSk0yefn4HU2fc6I5x1NNd94'
key = getpass.getpass('Password: ')
credentials = decryptCredentials(token,key)

#Setting up credentials for google maps api access
apiToken = b'gAAAAABaXyLsGnF3ms4sC3ZhoLCwWAx9q0tydWl8XKEwOy8CO0W6Eqc8J4om8HNDlNR9nExYCmSrelp8W5R-PLtcce1I2UgW3YnlXXqWvrMN-outYwXhZoc59djfF752mzOPqXBHgpNC'
apiKey = decryptString(apiToken,key)
gmaps.configure(api_key=apiKey)

# Connecting to database
try : 
    conn = db.connect("dbname={} user={} host={} password={}".format(credentials['database'],credentials['username'],credentials['host'],credentials['password']))
except :
    print("Unable to connect")
    
cursor = conn.cursor()    

Password: ········


### Refreshing stored procedures

This is necessary if the procedures defined in _procedures.py_ are modified.

In [3]:
# Refreshing stored procedures if necessary.
refresh = True
if refresh :
    refreshProcedures(credentials['database'],credentials['username'],credentials['host'],credentials['password'])

### Storing rivers in database

In [4]:
# Adding shapefile containing all of Norway's rivers    
#Shapefile info and schema.table where they will be stored
addRivers = False
riversShp = '/home/jose-luis/Documents/GeoData/RiversNVE/Elv_Elvenett.shp'
epsg_num = 3006 #epsg number for the shapefile, failed to obtain it programatically
schema = 'norway'
table = 'rivers'

rivers_cmd = "shp2pgsql -I -d -s {} {} {}.{}"
psql_cmd = "PGPASSWORD={} psql -U {} -d {} -h {} -p 5432 -q"

#The loadRivers() procedureas add a shapefile to the table using shp2pgsql
if addRivers :
        print("Loading all norwegian rivers...")
#        cursor.execute("SELECT procedures.loadRivers(%s,%s,%s,%s);", (riversShp, epsg_number, schema, table ) )
#        conn.commit()
        subprocess.check_call(rivers_cmd.format(epsg_num,riversShp, schema, table ) + ' | ' + \
                      psql_cmd.format(credentials['password'],credentials['username'],credentials['database'],credentials['host']), \
                      shell=True, stdout=open(os.devnull, 'wb') ) 
        print("Done!") 

### Storing dem's in database

Currently they should all be placed in a common folder. The projection is currently deduced from the file name, taking advantage of Kartverket naming convention (ends with _z32_ for UTM 32 and with _z33_ for UTM 33). 


In [5]:
addDEM = False
folderDEM = '/home/jose-luis/Documents/GeoData/DEM'
schema = 'norway'     
load_cmd = "raster2pgsql -I -C -M -b 1 -r -s {} -d -t 10x10 {}/*z{}.tif {}.{}"
psql_cmd = "PGPASSWORD={} psql -U {} -d {} -h {} -p 5432 -q"

if addDEM :
    print("Loading all elevation rasters in folder {}".format(folderDEM))
    table = 'demutm32'
    epsg_num = 32632
    subprocess.check_call(load_cmd.format(epsg_num, folderDEM, str(epsg_num)[-2:], schema, table ) + ' | ' + \
                          psql_cmd.format(credentials['password'],credentials['username'],credentials['database'],credentials['host']), \
                          shell=True, stdout=open(os.devnull, 'wb') ) 
    cursor.execute("SELECT procedures.setExtentTable(%s,%s);",(schema,epsg_num))
    conn.commit()
    table = 'demutm33'
    epsg_num = 32633
    subprocess.check_call(load_cmd.format(epsg_num,folderDEM,str(epsg_num)[-2:], schema, table ) + ' | ' + \
                      psql_cmd.format(credentials['password'],credentials['username'],credentials['database'],credentials['host']), \
                      shell=True, stdout=open(os.devnull, 'wb') ) 

    cursor.execute("SELECT procedures.setExtentTable(%s,%s);",(schema,epsg_num))
    conn.commit()
    print('Done!')

### Setting outlets for catchment delineation
As many catchment as there are outlets will be delineated. The outlets should be given in _.yaml_ file with the following format:



In [6]:
stationsFile = 'stations.yaml'

fid = open(stationsFile,'r')
code = fid.read()
fid.close
result = highlight(code, YamlLexer(),HtmlFormatter(linenos=False))
display(HTML(result))

The _.yaml_ file will be read and the above data stored as a table containing point geometries.

In [7]:
#Adding stations (coordinates) where the basins will be delineated
#Reading station data from a yaml file
stations = yaml.load(open(stationsFile))  
db.extras.register_composite('station_info',cursor)

#Re-arranging data as a list of tuples and passing it to pg with the help
#of pyscopg2 extras
allStations = list()
for i in stations:
    i=i['station']
    data = ( i['station_name'],
             i['station_id'],
             i['longitude'],
             i['latitude'],
             i['buffer'],
             i['epsg']
           )
    allStations.append(data)

cursor.execute("SELECT procedures.initializeStations();")
conn.commit()

cursor.execute("SELECT procedures.addStations( %s::station_info[] );",(allStations,))
conn.commit()



### Gathering data for catchment delineation
#### Initializing schema to store results

In [8]:
resultsSchema = 'basins'
cursor.execute("SELECT procedures.initializeResultsSchema( %s );",(resultsSchema,))
conn.commit()

#### Gathering dem from a circular buffer around the station

The Norway dem will be queried to get a the elevation in a circular buffer around the station. The size (radius) of the buffer is defined in the _.yaml_ file.

The rivers shapefile will be clipped according to the extent of the buffered dem and burned into it prior to catchment delineation.

The actual coordinates of the station will be modified so they fall on a river. The outlet will be set as the closest point between station coordinates and the rivers shapefile.

Please note that both rivers, dems, and stations might come in different coordinate systems. This is automatically handled only for some projections.

All the base data will be stored in a table.

In [9]:
# Getting raster around station

#Creating table to store results
dataTable = 'dem'
cursor.execute(" SELECT procedures.createDataTable(%s,%s,%s);", (resultsSchema,dataTable,32632))
conn.commit()
cursor.execute(" SELECT procedures.createDataTable(%s,%s,%s);", (resultsSchema,dataTable,32633))
conn.commit()

#Getting buffer raster around station
#Getting clipped rivers
#Burning-in rivers
#See procedures.py for details on the implementation
print("Loading base data...")
cursor.execute("SELECT procedures.generateBaseData(%s,%s);",(resultsSchema,32632));
cursor.execute("SELECT procedures.generateBaseData(%s,%s);",(resultsSchema,32633));
conn.commit()
print("Done!")

Loading base data...
Done!


### Catchment delineation

Currently the results will be stored for each different available dem projection and the table names generated dynamically based on the projection's epsg number.

In [10]:
#Creating tables to store results. One table per dem projection
resultsTable = 'results'
cursor.execute("SELECT procedures.createResultsTable(%s,%s,%s);",(resultsSchema,resultsTable,32632));
cursor.execute("SELECT procedures.createResultsTable(%s,%s,%s);",(resultsSchema,resultsTable,32633));
conn.commit()

#Creating folder to store intermediary results
tempDir = './Trash/'
if os.path.exists(tempDir) : 
    shutil.rmtree(tempDir)
os.mkdir(tempDir)

Preparing 
* _Taudem_ statements
* *gdal_translate* statements to get raster from database as _.tif_. This will be an input to taudem.
* _pgsql2psql_ to get each outlet as a shapefile that will be used to delineate the catchment by taudem.
* _raster2pgsql_ to upload the watershed raster to the results table. The catchment is delineated from this dem. 

In [11]:
#Creating strings for the taudem, raster2pgsql and shp2pgsql commands
#Getting dem with burned-in rivers from database
get_dem_cmd =           """gdal_translate -of GTiff PG:"host={} port='5432' dbname={} user={} password={} schema='{}' table='{}'  column='{}' where='{}' " {} """
#Filling dem
fill_cmd =              """mpiexec -n 8 pitremove -z {} -fel {}"""
#Computation of flow direction
flow_dir_cmd=           """mpiexec -n 8 d8flowdir -fel {} -p {}"""
#Computation of flow accumulation
flow_acc_cmd=           """mpiexec -n 8 aread8 -p {}  -nc -ad8 {}"""
#Getting outlet from database as a shapefile
station_as_shp_cmd =    """pgsql2shp -g outlet -f {} -h localhost -u {} -P {} {} "SELECT a.station_name, a.station_id, a.outlet FROM {} AS a WHERE a.station_id={}" """
#Delineating watershed
watershed_cmd=          """mpiexec -n 8 gagewatershed -p {} -o {} -gw {}"""
#Uploading watershed dem to database
rpg_cmd =               """raster2pgsql -b 1 -s {} -d {} {} | PGPASSWORD={} psql -U {} -d {} -h {} -p 5432"""

Processing watershed for a given dem projection

In [12]:
#Fetching all stations for a given projection
epsg_num = 32632
suffix = str(epsg_num)[-2:]
cursor.execute(sql.SQL(""" SELECT station_id,station_name FROM {}.{}; """)
                       .format(sql.Identifier(resultsSchema),
                               sql.Identifier(dataTable + suffix)
                               )
               );
conn.commit()
rows=cursor.fetchall()
tableName = 'results' + suffix
print("Processing stations for epsg {} ...".format(epsg_num))
for row in rows:
    sid = row[0]
    station_name = row[1]
    print(station_name)
    #Getting dem with burned rivers (falling within buffersize)
    subprocess.check_call(get_dem_cmd.format(credentials['host'],credentials['database'],credentials['username'],credentials['password'],
                                             'basins','dem' + suffix,'river_rast','station_id='+str(sid), tempDir + 'el.tif'), shell=True, stdout=open(os.devnull, 'wb'))
    #Filling dem
    subprocess.check_call(fill_cmd.format(tempDir + 'el.tif', tempDir + 'fel.tif'),                                     shell=True, stdout=open(os.devnull, 'wb'))
    #Get flow direction
    subprocess.check_call(flow_dir_cmd.format(tempDir + 'fel.tif', tempDir + 'd8.tif'),                                 shell=True, stdout=open(os.devnull, 'wb'))
    #Get flow accumulation
    subprocess.check_call(flow_acc_cmd.format(tempDir + 'd8.tif', tempDir + 'flow_acc.tif'),                            shell=True, stdout=open(os.devnull, 'wb'))
    #Getting station shapefile from postgis (necessary to specify the outlet in taudem)
    subprocess.check_call(station_as_shp_cmd.format(tempDir + 'station', credentials['username'], credentials['password'], credentials['database'],
                                                    'basins.dem' + suffix, sid),                                        shell=True, stdout=open(os.devnull, 'wb'))
    #Computing watershed for outlet
    subprocess.check_call(watershed_cmd.format(tempDir + 'd8.tif',tempDir + 'station.shp', tempDir + 'watershed.tif'),  shell=True, stdout=open(os.devnull, 'wb')) 
    #Uploading watershed raster to postgis
    tempTable = 'dummy'
    subprocess.check_call(rpg_cmd.format(str(epsg_num), tempDir + 'watershed.tif',resultsSchema + '.' + tempTable,
                                         credentials['password'],credentials['username'],
                                         credentials['database'],credentials['host']),                                  shell=True, stdout=open(os.devnull, 'wb')) 
    cursor.execute(sql.SQL(''' INSERT INTO {}.{}(station_id, station_name, rast)
                                         SELECT b.station_id, b.station_name, ST_MapAlgebra(a.rast, '1BB', '[rast]') 
                                         FROM (SELECT station_id,station_name FROM basins.stations) as b, (SELECT rast FROM {}.{}) AS a
                                         WHERE b.station_id=%s;
                               DROP TABLE {}.{};         
                              
                           '''
                           ).format(sql.Identifier(resultsSchema), sql.Identifier(tableName),
                                    sql.Identifier(resultsSchema), sql.Identifier(tempTable), 
                                    sql.Identifier(resultsSchema), sql.Identifier(tempTable) ),                                          
                   (sid,)
                  )
    conn.commit()
    

#Removing junk
[os.remove(os.path.join('.',f)) for f in os.listdir('.') if f.endswith(".tif")]    
    
#Changing data type to boolean for watershed raster
cursor.execute(sql.SQL('''  UPDATE {}.{}
                             SET basin = ST_Polygon(rast);
                            CREATE INDEX {} ON {}.{} USING GIST(basin);                         
                       ''').format(sql.Identifier(resultsSchema), sql.Identifier(tableName),
                                   sql.Identifier(tableName + '_idx'),
                                   sql.Identifier(resultsSchema),sql.Identifier(tableName)
                                  )           
               )
conn.commit();


cursor.execute(sql.SQL('''  UPDATE {}.{} as b
                             SET rast = ST_Clip(a.rast,basin)
                                        FROM {}.{} AS a 
                                        WHERE b.station_id = a.station_id;
                                                   
                       '''
                       ).format(sql.Identifier(resultsSchema), sql.Identifier(tableName),
                                   sql.Identifier(resultsSchema) ,sql.Identifier('dem' + suffix)
                               )           
               )
conn.commit();

print("Done!")

Processing stations for epsg 32632 ...
Langtjern, utløp
Birkenes
Øygardsbekken
Storgama v. dam
Kårvatn
Done!


### Visualization

The catchments will be displayed in google maps.


In [13]:
cursor.execute(""" SELECT json_build_object(
                    'type', 'FeatureCollection',

                    'features', json_agg(
                        json_build_object(
                            'type',       'Feature',
                            'label',       station_name,
                            'geometry',   ST_AsGeoJSON(ST_ForceRHR(st_transform(basin,4326)))::json,
                            'properties', jsonb_set(row_to_json(results32)::jsonb,'{basin}','0',false)
                             )
                        )
                   )
                    FROM basins.results32;
               """
              )


rows=cursor.fetchall()
fig = gmaps.figure()

for row in rows :
    basin_layer = gmaps.geojson_layer(row[0])
    fig.add_layer(basin_layer)
    
    
cursor.execute('''SELECT a.station_name, st_x(st_transform(a.outlet,4326)),
                  st_y(st_transform(a.outlet,4326)), st_area(b.basin)
                  FROM basins.dem32 AS a
                  INNER JOIN basins.results32 AS b 
                  ON a.station_id = b.station_id;''')  

rows=cursor.fetchall()

outlets = []
for row in rows:
    print(row)
    currentDict = {"name" : row[0], "location": (row[2],row[1]), "area": row[3]/1000000}
    outlets.append(currentDict)

outlet_locations = [outlet["location"] for outlet in outlets]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area}km2</dd>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(outlet_locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)


fig


('Langtjern, utløp', 9.72672740897311, 60.3724668119067, 4447400.0)
('Birkenes', 8.24164586793978, 58.3854634104195, 390900.0)
('Øygardsbekken', 6.10640047729555, 58.6219112793301, 1748000.0)
('Storgama v. dam', 8.65134235378551, 59.0501458751905, 227700.0)
('Kårvatn', 8.89355815133143, 62.7828341592326, 23762900.0)


Figure()

### Repeating catchment delineation for a different projection


In [14]:
#Creating folder to store intermediary results
tempDir = './Trash/'
if os.path.exists(tempDir) : 
    shutil.rmtree(tempDir)
os.mkdir(tempDir)

#Fetching all stations for a given projection
epsg_num = 32633
suffix = str(epsg_num)[-2:]
cursor.execute(sql.SQL(""" SELECT station_id,station_name FROM {}.{}; """)
                       .format(sql.Identifier(resultsSchema),
                               sql.Identifier(dataTable + suffix)
                               )
               );
conn.commit()
rows=cursor.fetchall()
tableName = 'results' + suffix
print("Processing stations for epsg {} ...".format(epsg_num))
for row in rows:
    sid = row[0]
    station_name = row[1]
    print(station_name)
    #Getting dem with burned rivers (falling within buffersize)
    subprocess.check_call(get_dem_cmd.format(credentials['host'],credentials['database'],credentials['username'],credentials['password'],
                                             'basins','dem' + suffix,'river_rast','station_id='+str(sid), tempDir + 'el.tif'), shell=True, stdout=open(os.devnull, 'wb'))
    #Filling dem
    subprocess.check_call(fill_cmd.format(tempDir + 'el.tif', tempDir + 'fel.tif'),                                     shell=True, stdout=open(os.devnull, 'wb'))
    #Get flow direction
    subprocess.check_call(flow_dir_cmd.format(tempDir + 'fel.tif', tempDir + 'd8.tif'),                                 shell=True, stdout=open(os.devnull, 'wb'))
    #Get flow accumulation
    subprocess.check_call(flow_acc_cmd.format(tempDir + 'd8.tif', tempDir + 'flow_acc.tif'),                            shell=True, stdout=open(os.devnull, 'wb'))
    #Getting station shapefile from postgis (necessary to specify the outlet in taudem)
    subprocess.check_call(station_as_shp_cmd.format(tempDir + 'station', credentials['username'], credentials['password'], credentials['database'],
                                                    'basins.dem' + suffix, sid),                                        shell=True, stdout=open(os.devnull, 'wb'))
    #Computing watershed for outlet
    subprocess.check_call(watershed_cmd.format(tempDir + 'd8.tif',tempDir + 'station.shp', tempDir + 'watershed.tif'),  shell=True, stdout=open(os.devnull, 'wb')) 
    #Uploading watershed raster to postgis
    tempTable = 'dummy'
    subprocess.check_call(rpg_cmd.format(str(epsg_num), tempDir + 'watershed.tif',resultsSchema + '.' + tempTable,
                                         credentials['password'],credentials['username'],
                                         credentials['database'],credentials['host']),                                  shell=True, stdout=open(os.devnull, 'wb')) 
    cursor.execute(sql.SQL(''' INSERT INTO {}.{}(station_id, station_name, rast)
                                         SELECT b.station_id, b.station_name, ST_MapAlgebra(a.rast, '1BB', '[rast]') 
                                         FROM (SELECT station_id,station_name FROM basins.stations) as b, (SELECT rast FROM {}.{}) AS a
                                         WHERE b.station_id=%s;
                               DROP TABLE {}.{};         
                              
                           '''
                           ).format(sql.Identifier(resultsSchema), sql.Identifier(tableName),
                                    sql.Identifier(resultsSchema), sql.Identifier(tempTable), 
                                    sql.Identifier(resultsSchema), sql.Identifier(tempTable) ),                                          
                   (sid,)
                  )
    conn.commit()
    

#Removing junk
[os.remove(os.path.join('.',f)) for f in os.listdir('.') if f.endswith(".tif")]    
    
#Changing data type to boolean for watershed raster
cursor.execute(sql.SQL('''  UPDATE {}.{}
                             SET basin = ST_Polygon(rast);
                            CREATE INDEX {} ON {}.{} USING GIST(basin);                         
                       ''').format(sql.Identifier(resultsSchema), sql.Identifier(tableName),
                                   sql.Identifier(tableName + '_idx'),
                                   sql.Identifier(resultsSchema),sql.Identifier(tableName)
                                  )           
               )
conn.commit();


cursor.execute(sql.SQL('''  UPDATE {}.{} as b
                             SET rast = ST_Clip(a.rast,basin)
                                        FROM {}.{} AS a 
                                        WHERE b.station_id = a.station_id;
                                                   
                       '''
                       ).format(sql.Identifier(resultsSchema), sql.Identifier(tableName),
                                   sql.Identifier(resultsSchema) ,sql.Identifier('dem' + suffix)
                               )           
               )
conn.commit();

print("Done!")

Processing stations for epsg 32633 ...
Birkenes
Øygardsbekken
Storgama v. dam
Dalelv
Done!


In [15]:
cursor.execute(""" SELECT json_build_object(
                    'type', 'FeatureCollection',

                    'features', json_agg(
                        json_build_object(
                            'type',       'Feature',
                            'label',       station_name,
                            'geometry',   ST_AsGeoJSON(ST_ForceRHR(st_transform(basin,4326)))::json,
                            'properties', jsonb_set(row_to_json(results33)::jsonb,'{basin}','0',false)
                             )
                        )
                   )
                    FROM basins.results33;
               """
              )


rows=cursor.fetchall()
fig = gmaps.figure()

for row in rows :
    basin_layer = gmaps.geojson_layer(row[0])
    fig.add_layer(basin_layer)
    
    
cursor.execute('''SELECT a.station_name, st_x(st_transform(a.outlet,4326)),
                  st_y(st_transform(a.outlet,4326)), st_area(b.basin)
                  FROM basins.dem33 AS a
                  INNER JOIN basins.results33 AS b 
                  ON a.station_id = b.station_id;''')  

rows=cursor.fetchall()

outlets = []
for row in rows:
    print(row)
    currentDict = {"name" : row[0], "location": (row[2],row[1]), "area": row[3]/1000000}
    outlets.append(currentDict)

outlet_locations = [outlet["location"] for outlet in outlets]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area}km2</dd>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(outlet_locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)


fig


('Birkenes', 8.2416458861021, 58.3854634023267, 389200.0)
('Øygardsbekken', 6.10640070158926, 58.6219112411965, 1760700.0)
('Storgama v. dam', 8.65134236427509, 59.0501458697299, 232100.0)
('Dalelv', 30.3854943737806, 69.6846717648601, 1435700.0)


Figure()